# Create all charts
---

## The data sets we used are:
* City of Austin, austin_311_service_requests.csv
* The Travis county zip codes from the US Census Bureau.
* Social/Economic census data for Austin, TX by zipcode from the US Census Bureau
* City of Austin Household statistics from the US Census Bureau.

In [9]:
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [7]:
path_311 = os.path.join('..', 'data', '311_cleaned_data_.csv')
df = pd.read_csv(path_311)
df.count()

service_request_id      275281
status_description      275281
mobile_web_status       275281
duplicate_verified      275281
override_duplicate      275281
functional_location     249932
source                  275280
closed_date             125670
updated_datetime        158982
open                    275281
closed                  275281
description             273914
agency_responsible      274720
service_name            274244
service_subtype          72326
lat                     275281
long                    275281
sap_problem_category    156983
sap_problem_type        156964
address                 274431
case_record_type        275281
district                274405
date                    275281
time                    275281
weekday                 275281
year                    275281
month                   275281
day                     275281
dtype: int64

### Number of 311 calls by month

In [17]:
df['month'].tail()

275276    7
275277    7
275278    7
275279    7
275280    7
Name: month, dtype: int64

In [12]:
months = df.groupby('month', as_index=False)['service_request_id'].count()
# months['month'] = months['month'].map(lambda x: datetime.date(1900, int(x), 1).strftime('%B'))
months

,month,service_request_id
0,7,275281


In [ ]:
plt.figure(figsize=[16, 8])
plt.bar(months['month'], months['service_request_id'])

### Number of 311 calls by type

In [ ]:
service_df = df.groupby('service_name')
service = service_df['service_request_id'].sum().sort_values()
fig, ax = plt.subplots(figsize=[16, 8])
ax.bar(service.index, service)
plt.xticks(rotation='90')
''

### breakdown of call categories if we have that information

### locations of callers (bubble plot?)

### Number of calls per capita by zipcode

### Number of calls by median household income of zip

### Number of calls by crime rate of zip

### Percent of call types by median household income (stacked bar)

### Number of calls by Temperature scatter

### Number fo calls by weekday